# Import Libraries

In [1]:
import gymnasium as gym
from typing import Tuple
from pathlib import Path
# 
from stable_baselines3 import PPO


In [2]:
import os
import shutil
import pandas as pd
from decimal import Decimal

from nautilus_trader.config import (
    LoggingConfig,
    CacheConfig,
    BacktestDataConfig,
    BacktestEngineConfig,
    BacktestRunConfig,
    BacktestVenueConfig,
    ImportableActorConfig,
    ImportableStrategyConfig,
    RiskEngineConfig,
    StreamingConfig,
)
# from nautilus_trader.backtest.node import BacktestDataConfig
# from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
# from nautilus_trader.backtest.node import BacktestRunConfig
# from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.backtest.models import FillModel

from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.objects import Quantity
from nautilus_trader.model.data import Bar
from nautilus_trader.model.data import BarType
from nautilus_trader.model.identifiers import TraderId

from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.persistence.catalog import ParquetDataCatalog # as DataCatalog

from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.test_kit.providers import CSVTickDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider


# Historical Data to Catalog

Some historical tick data was gotten from https://www.dukascopy.com/trading-tools/widgets/quotes/historical_data_feed 

In [3]:
# !apt-get update && apt-get install curl -y
# !curl https://raw.githubusercontent.com/nautechsystems/nautilus_data/main/nautilus_data/hist_data_to_catalog.py | python -

In [4]:
# from os import PathLike
# from pathlib import Path

# import requests
# from nautilus_trader.persistence.catalog import ParquetDataCatalog
# from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
# from nautilus_trader.test_kit.providers import CSVTickDataLoader
# from nautilus_trader.test_kit.providers import TestInstrumentProvider


# ROOT = Path(__file__).parent.parent
# CATALOG_DIR = ROOT / "catalog"
# CATALOG_DIR.mkdir(exist_ok=True)


# def load_fx_hist_data(
#     filename: str,
#     currency: str,
#     catalog_path: PathLike[str] | str,
# ) -> None:
#     instrument = TestInstrumentProvider.default_fx_ccy(currency)
#     wrangler = QuoteTickDataWrangler(instrument)

#     df = CSVTickDataLoader.load(
#         filename,
#         index_col=0,
#         datetime_format="%Y%m%d %H%M%S%f",
#     )
#     df.columns = ["bid_price", "ask_price", "size"]
#     print(df)

#     print("Preparing ticks...")
#     ticks = wrangler.process(df)

#     print("Writing data to catalog...")
#     catalog = ParquetDataCatalog(catalog_path)
#     catalog.write_data([instrument])
#     catalog.write_data(ticks)

#     print("Done")


# def download(url: str) -> None:
#     filename = url.rsplit("/", maxsplit=1)[1]
#     with open(filename, "wb") as f:
#         f.write(requests.get(url).content)


# def main():
#     # Download raw data
#     download(
#         "https://raw.githubusercontent.com/nautechsystems/nautilus_data/main/raw_data/fx_hist_data/DAT_ASCII_EURUSD_T_202001.csv.gz",
#     )
#     load_fx_hist_data(
#         filename="DAT_ASCII_EURUSD_T_202001.csv.gz",
#         currency="EUR/USD",
#         catalog_path=CATALOG_DIR,
#     )


# https://github.com/zcbmlijygrdwa/fx_EUR_USD_tick
# https://github.com/zeta-zetra/forexpy
# https://github.com/drui9/tickterial

# Load Data From Catalog

In [5]:
DATA_DIR = "../data"

path = Path(DATA_DIR).expanduser() / ""
raw_files = list(path.iterdir())
assert raw_files, f"Unable to find any histdata files in directory {path}"
raw_files

[PosixPath('../data/Coinbase_BTCUSD_1h.csv'),
 PosixPath('../data/Coinbase_BTCUSD_d.csv'),
 PosixPath('../data/configuration.json'),
 PosixPath('../data/configuration.yaml'),
 PosixPath('../data/EURUSD_2020-01-01.csv'),
 PosixPath('../data/EURUSD_Ticks_02.09.2024-02.09.2024.csv'),
 PosixPath('../data/EURUSD_Ticks_01.08.2024-01.08.2024.csv')]

In [6]:
raw_files[-1]

# df = pd.DataFrame(data=raw_files[-1].__str__(), columns=[])

PosixPath('../data/EURUSD_Ticks_01.08.2024-01.08.2024.csv')

In [7]:
# Here we just take the first data file found and load into a pandas DataFrame
df = CSVTickDataLoader.load(raw_files[-1], index_col='Local time', datetime_format="%Y-%m-%d %H:%M:%S.%f")
df

,Ask,Bid,AskVolume,BidVolume
Local time,,,,
2024-01-08 00:00:00.114000+01:00,1.08249,1.08246,6.30,3.6
2024-01-08 00:00:00.219000+01:00,1.08248,1.08246,0.90,3.6
2024-01-08 00:00:00.323000+01:00,1.08249,1.08247,0.90,3.6
2024-01-08 00:00:00.427000+01:00,1.08250,1.08246,1.80,5.4
2024-01-08 00:00:00.530000+01:00,1.08250,1.08247,1.80,3.6
...,...,...,...,...
2024-01-08 23:59:52.203000+01:00,1.07890,1.07886,5.49,3.6
2024-01-08 23:59:53.971000+01:00,1.07889,1.07885,5.49,4.5
2024-01-08 23:59:54.075000+01:00,1.07889,1.07884,5.49,5.4


In [8]:
df.iloc[-1]

Ask          1.07888
Bid          1.07884
AskVolume    3.60000
BidVolume    4.50000
Name: 2024-01-08 23:59:55.392000+01:00, dtype: float64

In [9]:
# df.drop(columns=['Unnamed: 0', 'Ask_Volume', 'Bid_Volume'], axis=1, inplace=True)
df.drop(columns=['AskVolume', 'BidVolume'], axis=1, inplace=True)
df

,Ask,Bid
Local time,,
2024-01-08 00:00:00.114000+01:00,1.08249,1.08246
2024-01-08 00:00:00.219000+01:00,1.08248,1.08246
2024-01-08 00:00:00.323000+01:00,1.08249,1.08247
2024-01-08 00:00:00.427000+01:00,1.08250,1.08246
2024-01-08 00:00:00.530000+01:00,1.08250,1.08247
...,...,...
2024-01-08 23:59:52.203000+01:00,1.07890,1.07886
2024-01-08 23:59:53.971000+01:00,1.07889,1.07885
2024-01-08 23:59:54.075000+01:00,1.07889,1.07884


In [10]:
df.index.set_names("timestamp", inplace=True)
# df.columns = ["bid_price", "ask_price"]
df.columns = ["ask_price", "bid_price"]
df

,ask_price,bid_price
timestamp,,
2024-01-08 00:00:00.114000+01:00,1.08249,1.08246
2024-01-08 00:00:00.219000+01:00,1.08248,1.08246
2024-01-08 00:00:00.323000+01:00,1.08249,1.08247
2024-01-08 00:00:00.427000+01:00,1.08250,1.08246
2024-01-08 00:00:00.530000+01:00,1.08250,1.08247
...,...,...
2024-01-08 23:59:52.203000+01:00,1.07890,1.07886
2024-01-08 23:59:53.971000+01:00,1.07889,1.07885
2024-01-08 23:59:54.075000+01:00,1.07889,1.07884


In [11]:
# df = df.reset_index()
# df

In [12]:
# Process quote ticks using a wrangler
EURUSD = TestInstrumentProvider.default_fx_ccy("EUR/USD")
wrangler = QuoteTickDataWrangler(EURUSD)

ticks = wrangler.process(df)
ticks

[QuoteTick(EUR/USD.SIM,1.08246,1.08249,1000000,1000000,1704668400114000000),
 QuoteTick(EUR/USD.SIM,1.08246,1.08248,1000000,1000000,1704668400219000000),
 QuoteTick(EUR/USD.SIM,1.08247,1.08249,1000000,1000000,1704668400323000000),
 QuoteTick(EUR/USD.SIM,1.08246,1.08250,1000000,1000000,1704668400427000000),
 QuoteTick(EUR/USD.SIM,1.08247,1.08250,1000000,1000000,1704668400530000000),
 QuoteTick(EUR/USD.SIM,1.08246,1.08250,1000000,1000000,1704668400633000000),
 QuoteTick(EUR/USD.SIM,1.08246,1.08249,1000000,1000000,1704668400838000000),
 QuoteTick(EUR/USD.SIM,1.08246,1.08250,1000000,1000000,1704668401041000000),
 QuoteTick(EUR/USD.SIM,1.08246,1.08249,1000000,1000000,1704668401349000000),
 QuoteTick(EUR/USD.SIM,1.08245,1.08249,1000000,1000000,1704668401802000000),
 QuoteTick(EUR/USD.SIM,1.08246,1.08249,1000000,1000000,1704668401905000000),
 QuoteTick(EUR/USD.SIM,1.08246,1.08250,1000000,1000000,1704668402209000000),
 QuoteTick(EUR/USD.SIM,1.08247,1.08250,1000000,1000000,1704668402515000000),

In [13]:
CATALOG_PATH = Path.cwd() / "catalog"

# Clear if it already exists, then create fresh
if CATALOG_PATH.exists():
    shutil.rmtree(CATALOG_PATH)
CATALOG_PATH.mkdir(parents=True)

# Create a catalog instance
catalog = ParquetDataCatalog(CATALOG_PATH)

# Write instrument to the catalog
catalog.write_data([EURUSD])

# Write ticks to catalog
catalog.write_data(ticks)

In [14]:
# You can also use a relative path such as `ParquetDataCatalog("./catalog")`,
# for example if you're running this notebook after the data setup from the docs.

# catalog = DataCatalog("./catalog")
# catalog = DataCatalog.from_env()
# catalog = ParquetDataCatalog("./catalog")
catalog.instruments()


[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

In [15]:
start = dt_to_unix_nanos(pd.Timestamp("2024-01-08", tz="UTC"))
end =  dt_to_unix_nanos(pd.Timestamp("2024-01-08T23", tz="UTC"))

# start = dt_to_unix_nanos(pd.Timestamp("2024-02-09", tz="UTC"))
# end =  dt_to_unix_nanos(pd.Timestamp("2024-10-09T23", tz="UTC"))

# start = dt_to_unix_nanos(pd.Timestamp("2020-01-03", tz="UTC"))
# end =  dt_to_unix_nanos(pd.Timestamp("2020-01-04", tz="UTC"))

catalog.quote_ticks(instrument_ids=[EURUSD.id.value], start=start, end=end)[:10]

[QuoteTick(EUR/USD.SIM,1.08235,1.08239,1000000,1000000,1704672000487000000),
 QuoteTick(EUR/USD.SIM,1.08235,1.08238,1000000,1000000,1704672000590000000),
 QuoteTick(EUR/USD.SIM,1.08234,1.08237,1000000,1000000,1704672000694000000),
 QuoteTick(EUR/USD.SIM,1.08234,1.08238,1000000,1000000,1704672000798000000),
 QuoteTick(EUR/USD.SIM,1.08234,1.08237,1000000,1000000,1704672000900000000),
 QuoteTick(EUR/USD.SIM,1.08234,1.08236,1000000,1000000,1704672001106000000),
 QuoteTick(EUR/USD.SIM,1.08232,1.08236,1000000,1000000,1704672001618000000),
 QuoteTick(EUR/USD.SIM,1.08232,1.08235,1000000,1000000,1704672001721000000),
 QuoteTick(EUR/USD.SIM,1.08233,1.08234,1000000,1000000,1704672002230000000),
 QuoteTick(EUR/USD.SIM,1.08233,1.08236,1000000,1000000,1704672002333000000)]

In [16]:
catalog.path

'/home/fortesenselabs/Tech/labs/Financial_Eng/Financial_Markets/lab/trade_flow/examples/notebooks/catalog'

In [17]:

instrument = catalog.instruments()[0]
instrument

CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)

# Set up Training Node


## Using High Level Backtest Node

In [18]:
# extend backtest node to TrainingNode
from nautilus_trader.backtest.config import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.objects import Money


os.environ['NAUTILUS_PATH'] = Path.cwd().__str__()
os.environ['NAUTILUS_PATH']

'/home/fortesenselabs/Tech/labs/Financial_Eng/Financial_Markets/lab/trade_flow/examples/notebooks'

### Add Venues

In [19]:
venue_configs = [
    BacktestVenueConfig(
        name="SIM",
        oms_type="HEDGING",
        account_type="MARGIN",  # Spot CASH account (not for perpetuals or futures)
        base_currency="USD",
        starting_balances=["1_000 USD"],
    ),
]

### Add Data

In [20]:
data_configs = [
    BacktestDataConfig(
        catalog_path=str(catalog.path), # str(ParquetDataCatalog.from_env().path)
        data_cls=QuoteTick,
        instrument_id=instrument.id,
        start_time=start,
        end_time=end,
    ),
]

### Add Strategies 

In [21]:
strategies = [
    ImportableStrategyConfig(
        strategy_path="nautilus_trader.examples.strategies.ema_cross:EMACross",
        config_path="nautilus_trader.examples.strategies.ema_cross:EMACrossConfig",
        config={
            "instrument_id": instrument.id,
            "bar_type": "EUR/USD.SIM-15-MINUTE-BID-INTERNAL", # EUR/USD.SIM-15-MINUTE-BID-INTERNAL
            "fast_ema_period": 10,
            "slow_ema_period": 20,
            "trade_size": Decimal(1_000),
        },
    ),
]

In [22]:
class TrainingNode(BacktestNode):
    pass

### Instantiate Node

In [23]:
config = BacktestRunConfig(
    engine=BacktestEngineConfig(strategies=strategies),
    data=data_configs,
    venues=venue_configs,
)

# TODO: check the backtest_high_level.ipynb notebook for reference on how to get this running

node = TrainingNode(configs=[config])

### Run Node

In [24]:
results = node.run()

2024-09-17T15:22:27.182947206Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2024-09-17T15:22:27.182969083Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2024-09-17T15:22:27.182972560Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2024-09-17T15:22:27.182975563Z [INFO] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2024. All rights reserved.
2024-09-17T15:22:27.182978862Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2024-09-17T15:22:27.182982740Z [INFO] BACKTESTER-001.BacktestEngine: 
2024-09-17T15:22:27.182985428Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2024-09-17T15:22:27.182989406Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2024-09-17T15:22:27.182991894Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀���
2024-09-17T15:

In [25]:
results

[BacktestResult(trader_id='BACKTESTER-001', machine_id='fortesense-hppro3500series', run_config_id='da3957891ff381bbc63c9017fb2d9b19b5858dae430abca156c747384a2f6c5f', instance_id='0dbc987b-4c3e-4c9d-aea2-362b182ed4f8', run_id='27df3a75-97ec-42fa-9da1-13340c6f51ca', run_started=1726586547483133000, run_finished=1726586549469016000, backtest_start=1704672000487000000, backtest_end=1704754795392000000, elapsed_time=82794.905, iterations=0, total_events=20, total_orders=10, total_positions=5, stats_pnls={'USD': {'PnL (total)': -0.55, 'PnL% (total)': -0.05499999999999545, 'Max Winner': 2.29, 'Avg Winner': np.float64(2.29), 'Min Winner': np.float64(2.29), 'Min Loser': np.float64(-0.01), 'Avg Loser': np.float64(-0.71), 'Max Loser': np.float64(-1.81), 'Expectancy': np.float64(-0.10999999999999993), 'Win Rate': 0.2}}, stats_returns={'Returns Volatility (252 days)': np.float64(nan), 'Average (Return)': np.float64(-6.598109824246617e-05), 'Average Loss (Return)': np.float64(-0.0008365339468216485

In [26]:
# results[0].generate_positions_report()

# Using Low Level Backtest Engine (low-level API)

Tutorial for [NautilusTrader](https://nautilustrader.io/docs/) a high-performance algorithmic trading platform and event driven backtester.

[View source on GitHub](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/getting_started/backtest_low_level.ipynb).

## Overview

This tutorial walks through how to use a `BacktestEngine` to backtest a simple EMA cross strategy
with a TWAP execution algorithm on a simulated Binance Spot exchange using historical trade tick data.

The following points will be covered:
- How to load raw data (external to Nautilus) using data loaders and wranglers
- How to add this data to a `BacktestEngine`
- How to add venues, strategies and execution algorithms to a `BacktestEngine`
- How to run backtests with a  `BacktestEngine`
- Post-run analysis and options for repeated runs

## Prerequisites
- Python 3.10+ installed
- [JupyterLab](https://jupyter.org/) or similar installed (`pip install -U jupyterlab`)
- [NautilusTrader](https://pypi.org/project/nautilus_trader/) latest release installed (`pip install -U nautilus_trader`)

## Imports

We'll start with all of our imports for the remainder of this tutorial.

In [27]:
from decimal import Decimal

from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.examples.algorithms.twap import TWAPExecAlgorithm
from nautilus_trader.examples.strategies.ema_cross_twap import EMACrossTWAP
from nautilus_trader.examples.strategies.ema_cross_twap import EMACrossTWAPConfig
from nautilus_trader.model.currencies import ETH
from nautilus_trader.model.currencies import USDT
from nautilus_trader.model.data import BarType
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.identifiers import TraderId
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.objects import Money
from nautilus_trader.persistence.wranglers import TradeTickDataWrangler
from nautilus_trader.test_kit.providers import TestDataProvider
from nautilus_trader.test_kit.providers import TestInstrumentProvider

## Loading data

For this tutorial we'll use some stub test data which exists in the NautilusTrader repository
(this data is also used by the automated test suite to test the correctness of the platform).

Firstly, instantiate a data provider which we can use to read raw CSV trade tick data into memory as a `pd.DataFrame`.
We then need to initialize the instrument which matches the data, in this case the `ETHUSDT` spot cryptocurrency pair for Binance.
We'll use this instrument for the remainder of this backtest run.

Next, we need to wrangle this data into a list of Nautilus `TradeTick` objects, which can we later add to the `BacktestEngine`.

In [28]:
# Load stub test data
provider = TestDataProvider()
trades_df = provider.read_csv_ticks("binance/ethusdt-trades.csv")

# Initialize the instrument which matches the data
ETHUSDT_BINANCE = TestInstrumentProvider.ethusdt_binance()

# Process into Nautilus objects
wrangler = TradeTickDataWrangler(instrument=ETHUSDT_BINANCE)
ticks = wrangler.process(trades_df)
ticks[:10]

Couldn't find test data directory, test data will be pulled from GitHub


[TradeTick(ETHUSDT.BINANCE,423.76,2.67900,SELLER,148568980,1597399200223000000),
 TradeTick(ETHUSDT.BINANCE,423.74,2.31976,SELLER,148568981,1597399200976000000),
 TradeTick(ETHUSDT.BINANCE,423.73,2.16924,SELLER,148568982,1597399200976000000),
 TradeTick(ETHUSDT.BINANCE,423.68,0.19096,SELLER,148568983,1597399201185000000),
 TradeTick(ETHUSDT.BINANCE,423.70,0.82490,BUYER,148568984,1597399201913000000),
 TradeTick(ETHUSDT.BINANCE,423.70,0.10117,BUYER,148568985,1597399202258000000),
 TradeTick(ETHUSDT.BINANCE,423.70,0.63290,BUYER,148568986,1597399202531000000),
 TradeTick(ETHUSDT.BINANCE,423.70,2.50000,BUYER,148568987,1597399203451000000),
 TradeTick(ETHUSDT.BINANCE,423.69,14.75000,SELLER,148568988,1597399204206000000),
 TradeTick(ETHUSDT.BINANCE,423.66,0.16914,BUYER,148568989,1597399204344000000)]

## Initialize a backtest engine

Now we'll need a backtest engine, minimally you could just call `BacktestEngine()` which will instantiate
an engine with a default configuration. 

Here we also show initializing a `BacktestEngineConfig` (will only a custom `trader_id` specified)
to show the general configuration pattern.

See the [Configuration](https://nautilustrader.io/docs/api_reference/config) API reference for details of all configuration options available.

In [29]:
# Configure backtest engine
config = BacktestEngineConfig(trader_id=TraderId("BACKTESTER-001"))

# Build the backtest engine
engine = BacktestEngine(config=config)

2024-09-17T15:22:35.198285671Z [INFO] BACKTESTER-001.BacktestEngine: Building system kernel
2024-09-17T15:22:35.198360271Z [INFO] BACKTESTER-001.MessageBus: config.database=None
2024-09-17T15:22:35.198386584Z [INFO] BACKTESTER-001.MessageBus: config.encoding='msgpack'
2024-09-17T15:22:35.198407417Z [INFO] BACKTESTER-001.MessageBus: config.timestamps_as_iso8601=False
2024-09-17T15:22:35.198427038Z [INFO] BACKTESTER-001.MessageBus: config.buffer_interval_ms=None
2024-09-17T15:22:35.198447915Z [INFO] BACKTESTER-001.MessageBus: config.autotrim_mins=None
2024-09-17T15:22:35.198469548Z [INFO] BACKTESTER-001.MessageBus: config.use_trader_prefix=True
2024-09-17T15:22:35.198488799Z [INFO] BACKTESTER-001.MessageBus: config.use_trader_id=True
2024-09-17T15:22:35.198509463Z [INFO] BACKTESTER-001.MessageBus: config.use_instance_id=False
2024-09-17T15:22:35.198531399Z [INFO] BACKTESTER-001.MessageBus: config.streams_prefix='stream'
2024-09-17T15:22:35.198552457Z [INFO] BACKTESTER-001.MessageBus: con

## Add venues

We'll need a venue to trade on, which should match the *market* data being added to the engine.

In this case we'll set up a *simulated* Binance Spot exchange.

In [30]:
# Add a trading venue (multiple venues possible)
BINANCE = Venue("BINANCE")
engine.add_venue(
    venue=BINANCE,
    oms_type=OmsType.NETTING,
    account_type=AccountType.CASH,  # Spot CASH account (not for perpetuals or futures)
    base_currency=None,  # Multi-currency account
    starting_balances=[Money(1_000_000.0, USDT), Money(10.0, ETH)],
)

2024-09-17T15:22:35.435786175Z [INFO] BACKTESTER-001.SimulatedExchange(BINANCE): OmsType=NETTING
2024-09-17T15:22:35.435860920Z [INFO] BACKTESTER-001.ExecClient-BINANCE: READY
2024-09-17T15:22:35.436037400Z [INFO] BACKTESTER-001.SimulatedExchange(BINANCE): Registered ExecutionClient-BINANCE
2024-09-17T15:22:35.436058090Z [INFO] BACKTESTER-001.ExecEngine: Registered ExecutionClient-BINANCE
2024-09-17T15:22:35.436076531Z [INFO] BACKTESTER-001.BacktestEngine: Added SimulatedExchange(id=BINANCE, oms_type=NETTING, account_type=CASH)


## Add data

Now we can add data to the backtest engine. First add the `Instrument` object we previously initialized, which matches our data.

Then we can add the trade ticks we wrangled earlier.

In [31]:
# Add instrument(s)
engine.add_instrument(ETHUSDT_BINANCE)

# Add data
engine.add_data(ticks)

2024-09-17T15:22:35.918057306Z [INFO] BACKTESTER-001.DataClient-BINANCE: READY
2024-09-17T15:22:35.918079198Z [INFO] BACKTESTER-001.DataEngine: Registered BINANCE
2024-09-17T15:22:35.918250091Z [INFO] BACKTESTER-001.SimulatedExchange(BINANCE): Added instrument ETHUSDT.BINANCE and created matching engine
2024-09-17T15:22:35.918254463Z [INFO] BACKTESTER-001.BacktestEngine: Added ETHUSDT.BINANCE Instrument
2024-09-17T15:22:35.927452700Z [INFO] BACKTESTER-001.BacktestEngine: Added 69,806 ETHUSDT.BINANCE TradeTick elements


:::note
The amount of and variety of data types is only limited by machine resources and your imagination (custom types are possible).
Also, multiple venues can be used for backtesting, again only limited by machine resources.
:::

## Add strategies

Now we can add the trading strategies we’d like to run as part of our system.

:::note
Multiple strategies and instruments can be used for backtesting, only limited by machine resources.
:::

Firstly, initialize a strategy configuration, then use this to initialize a strategy which we can add to the engine:

In [32]:
# Configure your strategy
strategy_config = EMACrossTWAPConfig(
    instrument_id=ETHUSDT_BINANCE.id,
    bar_type=BarType.from_str("ETHUSDT.BINANCE-250-TICK-LAST-INTERNAL"),
    trade_size=Decimal("0.10"),
    fast_ema_period=10,
    slow_ema_period=20,
    twap_horizon_secs=10.0,
    twap_interval_secs=2.5,
)

# Instantiate and add your strategy
strategy = EMACrossTWAP(config=strategy_config)
engine.add_strategy(strategy=strategy)

2024-09-17T15:22:36.110237026Z [INFO] BACKTESTER-001.EMACrossTWAP: READY
2024-09-17T15:22:36.110616457Z [INFO] BACKTESTER-001.ExecEngine: Registered OMS.UNSPECIFIED for Strategy EMACrossTWAP-000
2024-09-17T15:22:36.110622134Z [INFO] BACKTESTER-001.BACKTESTER-001: Registered Strategy EMACrossTWAP-000


You may notice that this strategy config includes parameters related to a TWAP execution algorithm.
This is because we can flexibly use different parameters per order submit, we still need to initialize
and add the actual `ExecAlgorithm` component which will execute the algorithm - which we'll do now.

## Add execution algorithms

NautilusTrader enables us to build up very complex systems of custom components. Here we show just one of the custom components
available, in this case a built-in TWAP execution algorithm. It is configured and added to the engine in generally the same pattern as for strategies:

:::note
Multiple execution algorithms can be used for backtesting, only limited by machine resources.
:::

In [33]:
# Instantiate and add your execution algorithm
exec_algorithm = TWAPExecAlgorithm()  # Using defaults
engine.add_exec_algorithm(exec_algorithm)

2024-09-17T15:22:36.439335860Z [INFO] BACKTESTER-001.TWAPExecAlgorithm: READY
2024-09-17T15:22:36.439356695Z [INFO] BACKTESTER-001.BACKTESTER-001: Registered ExecAlgorithm TWAP


## Run backtest

Now that we have our data, venues and trading system configured - we can run a backtest
Simply call the `.run(...)` method which will run a backtest over all available data by default.

See the [BacktestEngineConfig](https://nautilustrader.io/docs/latest/api_reference/config) API reference for a complete description of all available methods and options.

In [34]:
# Run the engine (from start to end of data)
engine.run()

2024-09-17T15:22:36.868126665Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=CASH, base_currency=None, is_reported=True, balances=[AccountBalance(total=1_000_000.00000000 USDT, locked=0.00000000 USDT, free=1_000_000.00000000 USDT), AccountBalance(total=10.00000000 ETH, locked=0.00000000 ETH, free=10.00000000 ETH)], margins=[], event_id=e2844c50-fc13-40b6-a6eb-3cbdfab9dd51)
2024-09-17T15:22:36.868147714Z [INFO] BACKTESTER-001.BacktestEngine: STARTING
2024-09-17T15:22:36.868154868Z [INFO] BACKTESTER-001.DataClient-BINANCE: Connecting...
2024-09-17T15:22:36.868159853Z [INFO] BACKTESTER-001.DataClient-BINANCE: Connected
2024-09-17T15:22:36.868163450Z [INFO] BACKTESTER-001.DataClient-BINANCE: RUNNING
2024-09-17T15:22:36.868166819Z [INFO] BACKTESTER-001.DataEngine: RUNNING
2024-09-17T15:22:36.868188428Z [INFO] BACKTESTER-001.RiskEngine: RUNNING
2024-09-17T15:22:36.868200563Z [INFO] BACKTESTER-001.ExecClient-BINANCE: Connecting...
2024-09-17T15:22:

## Post-run and analysis

Once the backtest is completed, a post-run tearsheet will be automatically logged using some
default statistics (or custom statistics which can be loaded, see the advanced [Portfolio statistics](../concepts/advanced/portfolio_statistics.md) guide).

Also, many resultant data and execution objects will be held in memory, which we
can use to further analyze the performance by generating various reports.

In [35]:
engine.trader.generate_account_report(BINANCE)

, position_id=ETHUSDT.BINANCE-EMACrossTWAP-000)
2020-08-14T13:49:36.198000000Z [INFO] BACKTESTER-001.EMACrossTWAP: <--[EVT] OrderSubmitted(instrument_id=ETHUSDT.BINANCE, client_order_id=O-20200814-134936-001-000-24, account_id=BINANCE-001, ts_event=1597412976198000000)
2020-08-14T13:49:36.198000000Z [INFO] BACKTESTER-001.EMACrossTWAP: <--[EVT] OrderInitialized(instrument_id=ETHUSDT.BINANCE, client_order_id=O-20200814-134936-001-000-25, side=BUY, type=MARKET, quantity=0.10000, time_in_force=FOK, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=TWAP, exec_algorithm_params={'horizon_secs': 10.0, 'interval_secs': 2.5}, exec_spawn_id=O-20200814-134936-001-000-25, tags=None)
2020-08-14T13:49:36.198000000Z [INFO] BACKTESTER-001.EMACrossTWAP: [CMD]--> SubmitOrder(order=MarketOrder(BUY 0.10000 ETHUSDT.B

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2020-08-14 10:00:00.223000+00:00,1000000.00000000,0E-8,1000000.00000000,USDT,BINANCE-001,CASH,None,[],True,{}
2020-08-14 10:00:00.223000+00:00,10.00000000,0E-8,10.00000000,ETH,BINANCE-001,CASH,None,[],True,{}
2020-08-14 10:22:34.574000+00:00,999989.38468857,0E-8,999989.38468857,USDT,BINANCE-001,CASH,None,[],False,{}
2020-08-14 10:22:34.574000+00:00,10.02500000,0E-8,10.02500000,ETH,BINANCE-001,CASH,None,[],False,{}
2020-08-14 10:22:37.074000+00:00,999978.76937714,0E-8,999978.76937714,USDT,BINANCE-001,CASH,None,[],False,{}
...,...,...,...,...,...,...,...,...,...,...
2020-08-14 14:24:40.817000+00:00,10.07500000,0E-8,10.07500000,ETH,BINANCE-001,CASH,None,[],False,{}
2020-08-14 14:24:43.317000+00:00,999956.20097362,0E-8,999956.20097362,USDT,BINANCE-001,CASH,None,[],False,{}
2020-08-14 14:24:43.317000+00:00,10.10000000,0E-8,10.10000000,ETH,BINANCE-001,CASH,None,[],False,{}
2020-08-14 14:59:58.693000+00:00,999998.88570472,0E-8,999998.88570472,USDT,BINANCE-001,CASH,None,[],False,{}


In [36]:
engine.trader.generate_order_fills_report()

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20200814-102234-001-000-1,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-1-004,ETHUSDT.BINANCE-EMACrossTWAP-000,BINANCE-001,BINANCE-1-004,MARKET,BUY,0.02500,...,None,None,None,TWAP,"{'horizon_secs': 10.0, 'interval_secs': 2.5}",O-20200814-102234-001-000-1,None,e5626299-832c-47b1-a3fc-bb8925d8bace,2020-08-14 10:22:34.574000+00:00,2020-08-14 10:22:42.074000+00:00
O-20200814-102234-001-000-1-E1,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-1-001,ETHUSDT.BINANCE-EMACrossTWAP-000,BINANCE-001,BINANCE-1-001,MARKET,BUY,0.02500,...,None,None,None,TWAP,None,O-20200814-102234-001-000-1,None,ad4f5938-b20f-41af-9499-c17a3e35ece3,2020-08-14 10:22:34.574000+00:00,2020-08-14 10:22:34.574000+00:00
O-20200814-102234-001-000-1-E2,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-1-002,ETHUSDT.BINANCE-EMACrossTWAP-000,BINANCE-001,BINANCE-1-002,MARKET,BUY,0.02500,...,None,None,None,TWAP,None,O-20200814-102234-001-000-1,None,37a7bc1f-1116-4988-8c52-bfc0d36612e2,2020-08-14 10:22:37.074000+00:00,2020-08-14 10:22:37.074000+00:00
O-20200814-102234-001-000-1-E3,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-1-003,ETHUSDT.BINANCE-EMACrossTWAP-000,BINANCE-001,BINANCE-1-003,MARKET,BUY,0.02500,...,None,None,None,TWAP,None,O-20200814-102234-001-000-1,None,19c340d9-3c87-45a4-921c-8d14ed1d69f9,2020-08-14 10:22:39.574000+00:00,2020-08-14 10:22:39.574000+00:00
O-20200814-103237-001-000-2,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-1-005,ETHUSDT.BINANCE-EMACrossTWAP-000,BINANCE-001,BINANCE-1-005,MARKET,SELL,0.10000,...,None,None,None,None,None,None,None,b6698eb1-e678-4184-95a3-8ecaf93a3766,2020-08-14 10:32:37.428000+00:00,2020-08-14 10:32:37.428000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20200814-142435-001-000-29,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-1-074,ETHUSDT.BINANCE-EMACrossTWAP-000,BINANCE-001,BINANCE-1-075,MARKET,BUY,0.02500,...,None,None,None,TWAP,"{'horizon_secs': 10.0, 'interval_secs': 2.5}",O-20200814-142435-001-000-29,None,fa0f7760-64f4-4b21-8757-aca1cee4a935,2020-08-14 14:24:35.817000+00:00,2020-08-14 14:24:43.317000+00:00
O-20200814-142435-001-000-29-E1,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-1-071,ETHUSDT.BINANCE-EMACrossTWAP-000,BINANCE-001,BINANCE-1-072,MARKET,BUY,0.02500,...,None,None,None,TWAP,None,O-20200814-142435-001-000-29,None,01bb7931-d867-47f3-840f-14c0b36c65fa,2020-08-14 14:24:35.817000+00:00,2020-08-14 14:24:35.817000+00:00
O-20200814-142435-001-000-29-E2,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-1-072,ETHUSDT.BINANCE-EMACrossTWAP-000,BINANCE-001,BINANCE-1-073,MARKET,BUY,0.02500,...,None,None,None,TWAP,None,O-20200814-142435-001-000-29,None,8c2001c7-7e7a-4fd3-b131-bc818c72fdbc,2020-08-14 14:24:38.317000+00:00,2020-08-14 14:24:38.317000+00:00


In [37]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,
ETHUSDT.BINANCE-EMACrossTWAP-000-e152262f-7774-4460-93b1-ff4c6c84a13c,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-001,O-20200814-102234-001-000-1-E1,O-20200814-103237-001-000-2,BUY,FLAT,0.00000,0.10000,2020-08-14 10:22:34.574000+00:00,1597401157428000000,2020-08-14 10:32:37.428000+00:00,602854000000,424.56249999999994,423.49,['0.00848054 USDT'],-0.00253,-0.11573054 USDT
ETHUSDT.BINANCE-EMACrossTWAP-000-ce48a4b8-27a1-4735-a06c-4a9f68b5c8d4,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-001,O-20200814-103237-001-000-3-E1,O-20200814-104611-001-000-4,SELL,FLAT,0.00000,0.10000,2020-08-14 10:32:37.428000+00:00,1597401971428000000,2020-08-14 10:46:11.428000+00:00,814000000000,423.48749999999995,424.7,['0.00848189 USDT'],-0.00286,-0.12973189 USDT
ETHUSDT.BINANCE-EMACrossTWAP-000-57d5464e-d78b-4bf2-bb08-2a47cceeaf25,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-001,O-20200814-104611-001-000-5-E1,O-20200814-110002-001-000-6,BUY,FLAT,0.00000,0.10000,2020-08-14 10:46:11.428000+00:00,1597402802097000000,2020-08-14 11:00:02.097000+00:00,830669000000,424.60499999999996,424.14398199999994,['0.00848750 USDT'],-0.00109,-0.05458930 USDT
ETHUSDT.BINANCE-EMACrossTWAP-000-8689f18e-b9e7-4388-aab0-b7a3df20404a,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-001,O-20200814-110002-001-000-7-E1,O-20200814-111402-001-000-8,SELL,FLAT,0.00000,0.10000,2020-08-14 11:00:02.097000+00:00,1597403642429000000,2020-08-14 11:14:02.429000+00:00,840332000000,424.00499999999994,425.57,['0.00849577 USDT'],-0.00369,-0.16499577 USDT
ETHUSDT.BINANCE-EMACrossTWAP-000-7c2d6404-019d-4715-a5ae-c0d63ccc174e,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-001,O-20200814-111402-001-000-9-E1,O-20200814-113300-001-000-10,BUY,FLAT,0.00000,0.10000,2020-08-14 11:14:02.429000+00:00,1597404780084000000,2020-08-14 11:33:00.084000+00:00,1137655000000,425.6375,425.34,['0.00850979 USDT'],-0.0007,-0.03825979 USDT
ETHUSDT.BINANCE-EMACrossTWAP-000-201da9db-b662-4ad1-9018-d662222f737a,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-001,O-20200814-113300-001-000-11-E1,O-20200814-113705-001-000-12,SELL,FLAT,0.00000,0.10000,2020-08-14 11:33:00.084000+00:00,1597405025204000000,2020-08-14 11:37:05.204000+00:00,245120000000,425.43499999999995,426.25,['0.00851685 USDT'],-0.00192,-0.09001685 USDT
ETHUSDT.BINANCE-EMACrossTWAP-000-4b86cc2f-ba1f-43ee-b874-33ef54fd1df6,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-001,O-20200814-113705-001-000-13-E1,O-20200814-120215-001-000-14,BUY,FLAT,0.00000,0.07500,2020-08-14 11:37:05.204000+00:00,1597406535235000000,2020-08-14 12:02:15.235000+00:00,1510031000000,426.30999999999995,426.43,['0.00639556 USDT'],0.00028,0.00260444 USDT
ETHUSDT.BINANCE-EMACrossTWAP-000-25c7ddaa-1a2e-4f68-a436-155a65f78d6e,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-001,O-20200814-120215-001-000-15-E1,O-20200814-123931-001-000-16,SELL,FLAT,0.00000,0.10000,2020-08-14 12:02:15.235000+00:00,1597408771933000000,2020-08-14 12:39:31.933000+00:00,2236698000000,426.36,427.33,['0.00853691 USDT'],-0.00228,-0.10553691 USDT
ETHUSDT.BINANCE-EMACrossTWAP-000-e60944ff-9def-41fc-aeb5-46dde11fcd5d,BACKTESTER-001,EMACrossTWAP-000,ETHUSDT.BINANCE,BINANCE-001,O-20200814-123931-001-000-17-E1,O-20200814-125346-001-000-18,BUY,FLAT,0.00000,0.10000,2020-08-14 12:39:31.933000+00:00,1597409626057000000,2020-08-14 12:53:46.057000+00:00,854124000000,427.32249999999993,426.26,['0.00853583 USDT'],-0.00249,-0.11478583 USDT


## Repeated runs

We can also choose to reset the engine for repeated runs with different strategy and component configurations.
Calling the `.reset(...)` method will retain all loaded data and components, but reset all other stateful values
as if we had a fresh `BacktestEngine` (this avoids having to load the same data again).

In [38]:
# For repeated backtest runs make sure to reset the engine
engine.reset()

2024-09-17T15:22:39.592902030Z [INFO] BACKTESTER-001.DataClient-BINANCE: READY
2024-09-17T15:22:39.592919302Z [INFO] BACKTESTER-001.DataEngine: READY
2024-09-17T15:22:39.592938441Z [INFO] BACKTESTER-001.ExecClient-BINANCE: READY
2024-09-17T15:22:39.593216580Z [INFO] BACKTESTER-001.Cache: Reset
2024-09-17T15:22:39.593232607Z [INFO] BACKTESTER-001.ExecEngine: READY
2024-09-17T15:22:39.593257468Z [INFO] BACKTESTER-001.RiskEngine: READY
2024-09-17T15:22:39.593279879Z [INFO] BACKTESTER-001.OrderEmulator: READY
2024-09-17T15:22:39.593327564Z [INFO] BACKTESTER-001.EMACrossTWAP: READY
2024-09-17T15:22:39.593352026Z [INFO] BACKTESTER-001.TWAPExecAlgorithm: READY
2024-09-17T15:22:39.593554217Z [INFO] BACKTESTER-001.Portfolio: READY
2024-09-17T15:22:39.593591012Z [INFO] BACKTESTER-001.BACKTESTER-001: READY
2024-09-17T15:22:39.593671183Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=CASH, base_currency=None, is_reported=True, balances=[AccountBalance(to

Individual components (actors, strategies, execution algorithms) need to be removed and added as required.

See the [Trader](../api_reference/trading.md) API reference for a description of all methods available to achieve this.

In [39]:
# Once done, good practice to dispose of the object if the script continues
engine.dispose()

2024-09-17T15:22:39.852427264Z [INFO] BACKTESTER-001.DataClient-BINANCE: DISPOSED
2024-09-17T15:22:39.852448931Z [INFO] BACKTESTER-001.DataEngine: DISPOSED
2024-09-17T15:22:39.852465006Z [INFO] BACKTESTER-001.RiskEngine: DISPOSED
2024-09-17T15:22:39.852501142Z [INFO] BACKTESTER-001.ExecClient-BINANCE: DISPOSED
2024-09-17T15:22:39.852523111Z [INFO] BACKTESTER-001.ExecEngine: DISPOSED
2024-09-17T15:22:39.852538628Z [INFO] BACKTESTER-001.MessageBus: Closed message bus
2024-09-17T15:22:39.852558381Z [INFO] BACKTESTER-001.BACKTESTER-001: Cleared all actors
2024-09-17T15:22:39.852585502Z [INFO] BACKTESTER-001.EMACrossTWAP: DISPOSED
2024-09-17T15:22:39.852603301Z [INFO] BACKTESTER-001.BACKTESTER-001: Cleared all trading strategies
2024-09-17T15:22:39.852628542Z [INFO] BACKTESTER-001.TWAPExecAlgorithm: DISPOSED
2024-09-17T15:22:39.852642154Z [INFO] BACKTESTER-001.BACKTESTER-001: Cleared all execution algorithms
2024-09-17T15:22:39.852661046Z [INFO] BACKTESTER-001.BACKTESTER-001: DISPOSED


# Create Agent

In [56]:
import datetime
from functools import partial
from typing import List, Optional

from nautilus_trader.core.datetime import nanos_to_secs
from nautilus_trader.model.data import Bar, BarType, BarSpecification
from nautilus_trader.model.enums import AggregationSource
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.data import DataType

from nautilus_trader.common.actor import Actor, ActorConfig
from nautilus_trader.common.enums import LogColor
from nautilus_trader.core.data import Data
from nautilus_trader.core.datetime import secs_to_nanos, unix_nanos_to_dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# OrderSideParser

import gymnasium as gym

from stable_baselines3 import PPO

In [57]:


from nautilus_trader.config import StrategyConfig
from nautilus_trader.core.data import Data
from nautilus_trader.core.datetime import unix_nanos_to_dt
from nautilus_trader.core.message import Event
from nautilus_trader.model.enums import OrderSide, PositionSide, TimeInForce
from nautilus_trader.model.events.position import (
    PositionChanged,
    PositionClosed,
    PositionEvent,
    PositionOpened,
)
from nautilus_trader.model.identifiers import InstrumentId, PositionId
from nautilus_trader.model.objects import Price, Quantity
from nautilus_trader.model.position import Position
from nautilus_trader.trading.strategy import Strategy

from nautilus_trader.model.functions import order_side_to_str

In [42]:
# Agents can extend actors 

class AgentConfig(ActorConfig):
    source_symbol: str
    target_symbol: str
    bar_spec: str = "10-SECOND-LAST"
    min_model_timedelta: str = "1D"


class Agent(Actor):
    def __init__(self, config: AgentConfig):
        super().__init__(config=config)

In [43]:
# 

In [44]:
# iterations = 200

# env = gym.make("CartPole-v1", render_mode="human")

# model = PPO("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=10_000)

# vec_env = model.get_env()
# obs = vec_env.reset()
# for i in range(iterations):
#     action, _states = model.predict(obs, deterministic=True)
#     obs, reward, done, info = vec_env.step(action)
#     vec_env.render()
#     # VecEnv resets automatically
#     # if done:
#     #   obs = env.reset()

# env.close()

## Supervised Learning

In [101]:
import yfinance as yf

,Dividends,Stock Splits
Date,,
2014-03-27 00:00:00-04:00,0.0,2.002000
2015-04-27 00:00:00-04:00,0.0,1.002746
2022-07-18 00:00:00-04:00,0.0,20.000000
2024-06-10 00:00:00-04:00,0.2,0.000000
2024-09-09 00:00:00-04:00,0.2,0.000000


In [105]:
smh_data = yf.download("SMH", start="2020-01-01", end="2021-01-01")
smh_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,71.894997,72.470001,71.654999,72.339996,70.230347,5200400
2020-01-03,71.275002,71.714996,70.940002,71.154999,69.079903,9963600
2020-01-06,70.254997,70.565002,69.885002,70.394997,68.342064,6514000
2020-01-07,71.120003,71.824997,70.699997,71.570000,69.482811,6526000
2020-01-08,71.565002,72.055000,71.260002,71.690002,69.599297,6112800
...,...,...,...,...,...,...
2020-12-24,106.599998,107.455002,106.599998,107.434998,105.028305,1142800
2020-12-28,108.639999,108.745003,107.120003,107.220001,104.818123,2445600
2020-12-29,107.839996,107.894997,105.959999,106.949997,104.554169,4497600


In [106]:
soxx_data = yf.download("SOXX", start="2020-01-01", end="2021-01-01")
soxx_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,84.753334,85.430000,84.333336,85.430000,81.918747,1275300
2020-01-03,84.113335,84.570000,83.653336,83.836670,80.390915,1235100
2020-01-06,82.783333,83.230003,82.419998,82.963333,79.553459,1615200
2020-01-07,83.996666,84.823334,83.500000,84.489998,81.017380,1146600
2020-01-08,84.656670,84.983330,84.053329,84.413330,80.943855,1297800
...,...,...,...,...,...,...
2020-12-24,123.930000,124.430000,123.556664,124.373337,120.611725,574200
2020-12-28,125.833336,125.900002,124.053329,124.236664,120.479195,1108500
2020-12-29,124.876663,124.876663,122.543335,123.916664,120.168869,1334700


### Pairs Trading Analysis

#### Premise
- Two (or more) assets prices are related in some way
- Their prices typically move together
- Opportunities arise when one assert deviates from the relationship
- Buy one, sell the other with the expectation they will return to their relationship at some point in the future

#### Benefits
- Applicable to a wide range of markets & assets
- market neutral (if the market suddenly tanks, don't lose money)

#### Challenges
- Modelling the relationship (drifting pairs)
- Transaction costs
- Getting "legged"

#### Example
- Two semiconductor ETFs `SMH` (VanEck Semiconductor ETF) and `SOXX` (iShares Semiconductor ETF)
- Fundamental reasons for a relationship (both ETFs of similar stocks)
- Visual analysis of price series confirms belief of relationship

In [99]:
def make_bar_type(instrument_id: InstrumentId, bar_spec) -> BarType:
    return BarType(instrument_id=instrument_id, bar_spec=bar_spec, aggregation_source=AggregationSource.EXTERNAL)


def one(iterable):
    if len(iterable) == 0:
        return None
    elif len(iterable) > 1:
        raise AssertionError("Too many values")
    else:
        return iterable[0]


def bars_to_dataframe(source_id: str, source_bars: List[Bar], target_id: str, target_bars: List[Bar]) -> pd.DataFrame:
    def _bars_to_frame(bars, instrument_id):
        df = pd.DataFrame([t.to_dict(t) for t in bars]).astype({"close": float})
        return df.assign(instrument_id=instrument_id).set_index(["instrument_id", "ts_init"])

    ldf = _bars_to_frame(bars=source_bars, instrument_id=source_id)
    rdf = _bars_to_frame(bars=target_bars, instrument_id=target_id)
    data = pd.concat([ldf, rdf])["close"].unstack(0).sort_index().fillna(method="ffill")
    return data.dropna()


def human_readable_duration(ns: float):
    from dateutil.relativedelta import relativedelta  # type: ignore

    seconds = nanos_to_secs(ns)
    delta = relativedelta(seconds=seconds)
    attrs = ["months", "days", "hours", "minutes", "seconds"]
    return ", ".join(
        [
            f"{getattr(delta, attr)} {attr if getattr(delta, attr) > 1 else attr[:-1]}"
            for attr in attrs
            if getattr(delta, attr)
        ]
    )


In [ ]:
import datetime
import pandas as pd
import hvplot.pandas
import holoviews as hv

# from demo.backtest import CATALOG as catalog
# from demo.util import bars_to_dataframe

In [ ]:
# Load pre-loaded sample data from nautilus DataCatalog
src_id = 'SMH.NASDAQ'
tgt_id = 'SOXX.NASDAQ'
src = catalog.bars(instrument_ids=[src_id], start=pd.Timestamp('2020-01-01'), end=pd.Timestamp('2020-01-10'), as_nautilus=True)
tgt = catalog.bars(instrument_ids=[tgt_id], start=pd.Timestamp('2020-01-01'), end=pd.Timestamp('2020-01-10'), as_nautilus=True)

In [ ]:
# Merge into single Dataframe for convenience, filter for market hours only
df = bars_to_dataframe(source_id=src_id, source_bars=src, target_id=tgt_id, target_bars=tgt)
df.index = pd.to_datetime(df.index)
df = df.between_time(datetime.time(14, 30), datetime.time(21,0))

In [ ]:
# View scatter plot of SMH vs SOXX
df.pct_change().cumsum().hvplot.step(y=[src_id, tgt_id], title=f"Time Series {src_id} vs {tgt_id}")

In [ ]:
# View scatter plot of SMH vs SOXX
df.hvplot.scatter(x=src_id, y=tgt_id, title=f"Price Scatter {src_id} vs {tgt_id}")

### Setup Nautilus Trader Node

#### Linear Regression Model

In [46]:
class ModelUpdate(Data):
    def __init__(
        self,
        model: LinearRegression,
        hedge_ratio: float,
        std_prediction: float,
        ts_init: int,
    ):
        super().__init__(ts_init=ts_init, ts_event=ts_init)
        self.model = model
        self.hedge_ratio = hedge_ratio
        self.std_prediction = std_prediction


class Prediction(Data):
    def __init__(
        self,
        instrument_id: str,
        prediction: float,
        ts_init: int,
    ):
        super().__init__(ts_init=ts_init, ts_event=ts_init)
        self.instrument_id = instrument_id
        self.prediction = prediction

class PredictedPriceConfig(ActorConfig):
    source_symbol: str
    target_symbol: str
    bar_spec: str = "10-SECOND-LAST"
    min_model_timedelta: str = "1D"



In [47]:

class PredictedPriceActor(Actor):
    def __init__(self, config: PredictedPriceConfig):
        super().__init__(config=config)

        self.source_id = InstrumentId.from_str(config.source_symbol)
        self.target_id = InstrumentId.from_str(config.target_symbol)
        self.bar_spec = BarSpecification.from_str(self.config.bar_spec)
        self.model: Optional[LinearRegression] = None
        self.hedge_ratio: Optional[float] = None
        self._min_model_timedelta = secs_to_nanos(pd.Timedelta(self.config.min_model_timedelta).total_seconds())
        self._last_model = pd.Timestamp(0)

    def on_start(self):
        # Set instruments
        self.left = self.cache.instrument(self.source_id)
        self.right = self.cache.instrument(self.target_id)

        # Subscribe to bars
        self.subscribe_bars(make_bar_type(instrument_id=self.source_id, bar_spec=self.bar_spec))
        self.subscribe_bars(make_bar_type(instrument_id=self.target_id, bar_spec=self.bar_spec))

    def on_bar(self, bar: Bar):
        self._check_model_fit(bar)
        self._predict(bar)

    @property
    def data_length_valid(self) -> bool:
        return self._check_first_tick(self.source_id) and self._check_first_tick(self.target_id)

    @property
    def has_fit_model_today(self):
        return unix_nanos_to_dt(self.clock.timestamp_ns()).date() == self._last_model.date()

    def _check_first_tick(self, instrument_id) -> bool:
        """Check we have enough bar data for this `instrument_id`, according to `min_model_timedelta`"""
        bars = self.cache.bars(bar_type=make_bar_type(instrument_id, bar_spec=self.bar_spec))
        if not bars:
            return False
        delta = self.clock.timestamp_ns() - bars[-1].ts_init
        return delta > self._min_model_timedelta

    def _check_model_fit(self, bar: Bar):
        # Check we have the minimum required data
        if not self.data_length_valid:
            return

        # Check we haven't fit a model yet today
        if self.has_fit_model_today:
            return

        # Generate a dataframe from cached bar data
        df = bars_to_dataframe(
            source_id=self.source_id.value,
            source_bars=self.cache.bars(bar_type=make_bar_type(self.source_id, bar_spec=self.bar_spec)),
            target_id=self.target_id.value,
            target_bars=self.cache.bars(bar_type=make_bar_type(self.target_id, bar_spec=self.bar_spec)),
        )

        # Format the arrays for scikit-learn
        X = df.loc[:, self.source_id.value].astype(float).values.reshape(-1, 1)
        Y = df.loc[:, self.target_id.value].astype(float).values.reshape(-1, 1)

        # Fit a model
        self.model = LinearRegression(fit_intercept=False)
        self.model.fit(X, Y)
        self.log.info(
            f"Fit model @ {unix_nanos_to_dt(bar.ts_init)}, r2: {r2_score(Y, self.model.predict(X))}",
            color=LogColor.BLUE,
        )
        self._last_model = unix_nanos_to_dt(bar.ts_init)

        # Record std dev of predictions (used for scaling our order price)
        pred = self.model.predict(X)
        errors = pred - Y
        std_pred = errors.std()

        # The model slope is our hedge ratio (the ratio of source
        self.hedge_ratio = float(self.model.coef_[0][0])
        self.log.info(f"Computed hedge_ratio={self.hedge_ratio:0.4f}", color=LogColor.BLUE)

        # Publish model
        model_update = ModelUpdate(
            model=self.model, hedge_ratio=self.hedge_ratio, std_prediction=std_pred, ts_init=bar.ts_init
        )
        self.publish_data(
            data_type=DataType(ModelUpdate, metadata={"instrument_id": self.target_id.value}), data=model_update
        )

    def _predict(self, bar: Bar):
        if self.model is not None and bar.bar_type.instrument_id == self.source_id:
            pred = self.model.predict([[bar.close]])[0][0]
            prediction = Prediction(instrument_id=self.target_id, prediction=pred, ts_init=bar.ts_init)
            self.publish_data(
                data_type=DataType(Prediction, metadata={"instrument_id": self.target_id.value}), data=prediction
            )


#### Strategy

In [48]:
class PairTraderConfig(StrategyConfig):
    source_symbol: str
    target_symbol: str
    notional_trade_size_usd: int = 10_000
    min_model_timedelta: datetime.timedelta = datetime.timedelta(days=1)
    trade_width_std_dev: float = 2.5
    bar_spec: str = "10-SECOND-LAST"
    ib_long_short_margin_requirement = (0.25 + 0.17) / 2.0


In [58]:
class PairTrader(Strategy):
    def __init__(self, config: PairTraderConfig):
        super().__init__(config=config)
        self.source_id = InstrumentId.from_str(config.source_symbol)
        self.target_id = InstrumentId.from_str(config.target_symbol)
        self.model: Optional[ModelUpdate] = None
        self.hedge_ratio: Optional[float] = None
        self.std_pred: Optional[float] = None
        self.prediction: Optional[float] = None
        self._current_edge: float = 0.0
        self._current_required_edge: float = 0.0
        self.bar_spec = BarSpecification.from_str(self.config.bar_spec)
        self._summarised: set = set()
        self._position_id: int = 0

    def on_start(self):
        # Set instruments
        self.source = self.cache.instrument(self.source_id)
        self.target = self.cache.instrument(self.target_id)

        # Subscribe to bars
        self.subscribe_bars(make_bar_type(instrument_id=self.source_id, bar_spec=self.bar_spec))
        self.subscribe_bars(make_bar_type(instrument_id=self.target_id, bar_spec=self.bar_spec))

        # Subscribe to model and predictions
        self.subscribe_data(data_type=DataType(ModelUpdate, metadata={"instrument_id": self.target_id.value}))
        self.subscribe_data(data_type=DataType(Prediction, metadata={"instrument_id": self.target_id.value}))

    def on_bar(self, bar: Bar):
        self._update_theoretical()
        self._check_for_entry(bar)
        self._check_for_exit(timer=None, bar=bar)

    def on_data(self, data: Data):
        if isinstance(data, ModelUpdate):
            self._on_model_update(data)
        elif isinstance(data, Prediction):
            self._on_prediction(data)
        else:
            raise TypeError()

    def on_event(self, event: Event):
        self._check_for_hedge(timer=None, event=event)
        if isinstance(event, (PositionOpened, PositionChanged)):
            position = self.cache.position(event.position_id)
            self._log.info(f"{position}", color=LogColor.YELLOW)
            assert position.quantity < 200  # Runtime check for bug in code

    def _on_model_update(self, model_update: ModelUpdate):
        self.model = model_update.model
        self.hedge_ratio = model_update.hedge_ratio
        self.std_pred = model_update.std_prediction

    def _on_prediction(self, prediction: Prediction):
        self.prediction = prediction.prediction
        self._update_theoretical()

    def _update_theoretical(self):
        """We've either received an update Bar market or a new prediction, update our `current_edge`"""
        if not self.prediction:
            return

        quote_right: Bar = self.cache.bar(make_bar_type(self.target_id, bar_spec=self.bar_spec))
        if not quote_right:
            return

        self._current_edge = 0
        close_target = quote_right.close
        if (self.prediction - close_target) > 0:
            self._current_edge = self.prediction - close_target
        elif (close_target - self.prediction) > 0:
            self._current_edge = close_target - self.prediction

    def _check_for_entry(self, bar: Bar):
        if bar.bar_type.instrument_id == self.target_id and self.prediction is not None:
            # Send in orders
            quote_target: Bar = self.cache.bar(make_bar_type(self.target_id, bar_spec=self.bar_spec))
            if not quote_target:
                return

            market_right = quote_target.close
            self._current_required_edge = self.std_pred * self.config.trade_width_std_dev

            if self._current_edge > self._current_required_edge:
                # Our theoretical price is above the market; we want to buy
                side = OrderSide.BUY
                max_volume = int(self.config.notional_trade_size_usd / market_right)
                capped_volume = self._cap_volume(instrument_id=self.target_id, max_quantity=max_volume)
                price = self.prediction - self._current_required_edge
                self._log.debug(f"{side} {max_volume=} {capped_volume=} {price=}")
            elif self._current_edge < -self._current_required_edge:
                # Our theoretical price is below the market; we want to sell
                side = OrderSide.SELL
                max_volume = int(self.config.notional_trade_size_usd / market_right)
                capped_volume = self._cap_volume(instrument_id=self.target_id, max_quantity=max_volume)
                price = self.prediction + self._current_required_edge
                self._log.debug(f"{side} {max_volume=} {capped_volume=} {price=}")
            else:
                return
            if capped_volume == 0:
                # We're at our max limit, cancel any remaining orders and return
                for order in self.cache.orders_open(instrument_id=self.target_id, strategy_id=self.id):
                    self.cancel_order(order=order)
                return
            self._log.info(
                f"Entry opportunity: {side} market={market_right}, "
                f"theo={self.prediction:0.3f} {capped_volume=} ({self._current_edge=:0.3f}, "
                f"{self._current_required_edge=:0.3f})",
                color=LogColor.GREEN,
            )
            # Cancel any existing orders
            for order in self.cache.orders_open(instrument_id=self.target_id, strategy_id=self.id):
                self.cancel_order(order=order)
            order = self.order_factory.limit(
                instrument_id=self.target_id,
                order_side=side,
                price=Price(price, self.target.price_precision),
                quantity=Quantity.from_int(capped_volume),
                time_in_force=TimeInForce.IOC,
            )
            self._log.info(f"ENTRY {order.info()}", color=LogColor.BLUE)
            self.submit_order(order, PositionId(f"target-{self._position_id}"))

    def _cap_volume(self, instrument_id: InstrumentId, max_quantity: int) -> int:
        position_quantity = 0
        position = self.current_position(instrument_id)
        if position is not None:
            position_quantity = position.quantity
        return max(0, max_quantity - position_quantity)

    def _check_for_hedge(self, timer=None, event: Optional[Event] = None):
        if not ((isinstance(event, (PositionEvent,)) and event.instrument_id == self.target_id)):
            return

        timer_name = f"hedge-{self.id}"
        try:
            self._hedge_position(event)
            # Keep scheduling this method to run until we're hedged
            if timer_name in self.clock.timer_names:
                self.clock.cancel_timer(timer_name)
            self.clock.set_time_alert(
                name=timer_name,
                alert_time=self.clock.utc_now() + pd.Timedelta(seconds=2),
                callback=partial(self._check_for_hedge, event=event),
            )
        except RepeatedEventComplete:
            # Hedge is complete, return
            if timer_name in self.clock.timer_names:
                self.clock.cancel_timer(timer_name)
            return

    def _hedge_position(self, event: PositionEvent):
        # We've opened or changed position in our source instrument, we will likely need to hedge.
        target_position = self.cache.position(event.position_id)
        hedge_quantity = int(round(target_position.quantity * self.hedge_ratio, 0))
        quantity = 0
        if isinstance(event, PositionClosed):
            # (possibly) Reducing our position in the target instrument
            source_position: Position = self.current_position(self.source_id)
            if source_position is not None and source_position.is_closed:
                if source_position.id.value not in self._summarised:
                    self._summarise_position()
                    self._position_id += 1
                quantity = source_position.quantity
                side = self._opposite_side(source_position.side)
        else:
            # (possibly) Increasing our position in hedge instrument
            side = self._opposite_side(target_position.side)
            quantity = self._cap_volume(instrument_id=self.source_id, max_quantity=hedge_quantity)

        if quantity == 0:
            # Fully hedged, cancel any existing orders
            for order in self.cache.orders_open(instrument_id=self.source_id, strategy_id=self.id):
                self.cancel_order(order=order)
            raise RepeatedEventComplete
        elif self.cache.orders_inflight(instrument_id=self.source_id, strategy_id=self.id):
            # Don't send more orders if we have some currently in-flight
            return

        # Cancel any existing orders
        for order in self.cache.orders_open(instrument_id=self.source_id, strategy_id=self.id):
            self.cancel_order(order=order)
        order = self.order_factory.market(
            instrument_id=self.source_id,
            order_side=side,
            quantity=Quantity.from_int(quantity),
        )
        self._log.info(f"ENTRY HEDGE {order.info()}", color=LogColor.BLUE)
        self.submit_order(order, PositionId(f"source-{self._position_id}"))
        return order

    def _check_for_exit(self, timer=None, bar: Optional[Bar] = None):
        if not self.cache.positions(strategy_id=self.id):
            return

        # Keep checking that we have successfully got a hedge
        timer_name = f"exit-{self.id}"
        try:
            self._exit_position(bar=bar)
            # Keep scheduling this method to run until we're exited
            if timer_name in self.clock.timer_names:
                self.clock.cancel_timer(timer_name)
            self.clock.set_time_alert(
                name=timer_name,
                alert_time=self.clock.utc_now() + pd.Timedelta(seconds=2),
                callback=partial(self._check_for_exit, bar=bar),
            )
        except RepeatedEventComplete:
            # Hedge is complete, return
            if timer_name in self.clock.timer_names:
                self.clock.cancel_timer(timer_name)
            return

    def _exit_position(self, bar: Bar):
        position: Position = self.current_position(self.target_id)
        if position is not None:
            if position.is_closed:
                raise RepeatedEventComplete()
            if self._current_edge < (self._current_required_edge * 0.25):
                if self.cache.orders_inflight(instrument_id=self.target_id, strategy_id=self.id):
                    # Order currently in-flight, don't send again
                    return
                self._log.info(
                    f"Trigger to close position {self._current_edge=:0.3f} {self._current_required_edge=:0.3f} (* 0.25)",
                    color=LogColor.CYAN,
                )
                # We're close back to fair value, we should try and close our position
                order = self.order_factory.market(
                    instrument_id=self.target_id,
                    order_side=self._opposite_side(position.side),
                    quantity=position.quantity,
                )
                self._log.info(f"CLOSE {order.info()}", color=LogColor.BLUE)
                self.submit_order(order, PositionId(f"target-{self._position_id}"))

    def current_position(self, instrument_id: InstrumentId) -> Optional[Position]:
        try:
            side = {self.source_id: "source", self.target_id: "target"}[instrument_id]
            return self.cache.position(PositionId(f"{side}-{self._position_id}"))
        except AssertionError:
            return None

    def _opposite_side(self, side: PositionSide):
        return {PositionSide.LONG: OrderSide.SELL, PositionSide.SHORT: OrderSide.BUY, PositionSide.FLAT: None}[side]

    def _summarise_position(self):
        src_pos: Position = self.current_position(instrument_id=self.source_id)
        tgt_pos: Position = self.current_position(instrument_id=self.target_id)
        self.log.warning("Hedge summary:", color=LogColor.BLUE)
        self.log.warning(
            f"target: {order_side_to_str(tgt_pos.events[0].order_side)} {tgt_pos.peak_qty}, "
            f"{tgt_pos.avg_px_open=}, {tgt_pos.avg_px_close=}, {tgt_pos.realized_return=:0.4f}",
            color=LogColor.NORMAL,
        )
        self.log.warning(
            f"source: {order_side_to_str(src_pos.events[0].order_side)} {src_pos.peak_qty}, "
            f"{src_pos.avg_px_open=}, {src_pos.avg_px_close=}, {src_pos.realized_return=:0.4f}",
            color=LogColor.NORMAL,
        )

        def peak_notional(pos):
            entry_order = self.cache.order(pos.events[0].client_order_id)
            return pos.peak_qty * {OrderSide.BUY: 1.0, OrderSide.SELL: -1.0}[entry_order.side] * pos.avg_px_open

        tgt_notional = peak_notional(tgt_pos)
        src_notional = peak_notional(src_pos)
        margin_requirements = (abs(tgt_notional) + abs(src_notional)) * self.config.ib_long_short_margin_requirement
        pnl = src_pos.realized_pnl + tgt_pos.realized_pnl
        return_bps = float(pnl) / margin_requirements * 10_000
        self.log.warning(
            f"position duration = {human_readable_duration(src_pos.duration_ns)} "
            f"(opened={unix_nanos_to_dt(src_pos.ts_opened)}, closed={unix_nanos_to_dt(src_pos.ts_closed)}",
            color=LogColor.NORMAL,
        )
        self.log.warning(
            f"Spread=({tgt_notional:.0f}/{src_notional:.0f}), total_margin_required={margin_requirements:0.1f} "
            f"PNL=${pnl}, margin_return={return_bps:0.1f}bps\n",
            color=LogColor.GREEN if pnl > 0 else LogColor.RED,
        )

        self._summarised.add(src_pos.id.value)

    def on_stop(self):
        self.close_all_positions(self.source_id)
        self.close_all_positions(self.target_id)


class RepeatedEventComplete(Exception):
    pass


#### Model Backtest

In [50]:
import pathlib
from typing import Tuple

from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.config import (
    CacheConfig,
    BacktestDataConfig,
    BacktestEngineConfig,
    BacktestRunConfig,
    BacktestVenueConfig,
    ImportableActorConfig,
    ImportableStrategyConfig,
    RiskEngineConfig,
    StreamingConfig,
)
# from nautilus_trader.persistence.catalog import ParquetDataCatalog as DataCatalog


#### Load Data

In [64]:
# nautilus_talks_catalog_path = str(pathlib.Path.cwd().parent.joinpath("nautilus_talks/20220617/demo/catalog"))
nautilus_talks_catalog_path = str(pathlib.Path.cwd().joinpath("catalog"))
nautilus_talks_catalog_path

'/home/fortesenselabs/Tech/labs/Financial_Eng/Financial_Markets/lab/trade_flow/examples/notebooks/catalog'

In [65]:

os.listdir(nautilus_talks_catalog_path)

['data']

In [70]:

# CATALOG = DataCatalog(str(pathlib.Path(__file__).parent.joinpath("catalog")))
LR_MODEL_DATA_CATALOG = ParquetDataCatalog(nautilus_talks_catalog_path)

LR_MODEL_DATA_CATALOG.instruments()

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

#### Run Model Backtest

In [97]:

def main(
    instrument_ids: Tuple[str, str],
    catalog: ParquetDataCatalog,
    notional_trade_size_usd: int = 10_000,
    start_time: str = None,
    end_time: str = None,
    log_level: str = "ERROR",
    bypass_logging: bool = False,
    persistence: bool = False,
    **strategy_kwargs,
):
    # Create model prediction actor
    prediction = ImportableActorConfig(
        actor_path="model:PredictedPriceActor",
        config_path="model:PredictedPriceConfig",
        config=dict(
            source_symbol=instrument_ids[0],
            target_symbol=instrument_ids[1],
        ),
    )

    # Create strategy
    strategy = ImportableStrategyConfig(
        strategy_path="strategy:PairTrader",
        config_path="strategy:PairTraderConfig",
        config=dict(
            source_symbol=instrument_ids[0],
            target_symbol=instrument_ids[1],
            notional_trade_size_usd=notional_trade_size_usd,
            **strategy_kwargs,
        ),
    )

    # Create backtest engine
    engine = BacktestEngineConfig(
        trader_id="BACKTESTER-001",
        cache=CacheConfig(tick_capacity=100_000),
        # bypass_logging=bypass_logging,
        # log_level=log_level,
        streaming=StreamingConfig(catalog_path=str(catalog.path)) if persistence else None,
        risk_engine=RiskEngineConfig(max_order_submit_rate="1000/00:00:01"),  # type: ignore
        strategies=[strategy],
        actors=[prediction],
    )
    venues = [
        BacktestVenueConfig(
            name="SIM", # "NASDAQ"
            oms_type="NETTING",
            account_type="CASH",
            base_currency="USD",
            starting_balances=["1_000_000 USD"],
        )
    ]
    print("instrument_ids => ", instrument_ids)
    data = [
        BacktestDataConfig(
            data_cls=Bar.fully_qualified_name(),
            catalog_path=str(catalog.path),
            catalog_fs_protocol=catalog.fs_protocol,
            catalog_fs_storage_options=catalog.fs_storage_options,
            instrument_id=InstrumentId.from_str(instrument_id),
            start_time=start_time,
            end_time=end_time,
        )
        for instrument_id in instrument_ids
    ]

    run_config = BacktestRunConfig(engine=engine, venues=venues, data=data)
    
    print("venues => ", run_config.venues)
    node = BacktestNode(configs=[run_config])
    return node.run()


In [98]:
# typer.run(main)
# lr_catalog = LR_MODEL_DATA_CATALOG

assert len(LR_MODEL_DATA_CATALOG.instruments())>0, "Couldn't load instruments, have you run `poetry run inv extract-catalog`?"
    
[result] = main(
        catalog=LR_MODEL_DATA_CATALOG,
        # instrument_ids=("SMH.NASDAQ", "SOXX.NASDAQ"),
        instrument_ids=("EURUSD.SIM"),
        log_level="INFO",
        persistence=False,
        end_time="2020-06-01",
    )

print(result.instance_id)


instrument_ids =>  EURUSD.SIM


ValueError: Error parsing `InstrumentId` from 'E': Missing '.' separator between symbol and venue components

# DUMP

## Add simulation module

We can optionally plug in a module to simulate rollover interest. The data is available from pre-packaged test data.

In [ ]:
from nautilus_trader.backtest.modules import FXRolloverInterestConfig
from nautilus_trader.backtest.modules import FXRolloverInterestModule
from nautilus_trader.test_kit.providers import TestDataProvider

provider = TestDataProvider()
interest_rate_data = provider.read_csv("../data/short-term-interest.csv")
config = FXRolloverInterestConfig(interest_rate_data)
fx_rollover_interest = FXRolloverInterestModule(config=config)

## Add fill model

For this backtest we'll use a simple probabilistic fill model.

In [ ]:
fill_model = FillModel(
    prob_fill_on_limit=0.2,
    prob_fill_on_stop=0.95,
    prob_slippage=0.5,
    random_seed=42,
)

## Add venue

For this backtest we just need a single trading venue which will be a similated FX ECN.

In [ ]:
from nautilus_trader.model.currencies import JPY
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.objects import Money

SIM = Venue("SIM")
engine.add_venue(
    venue=SIM,
    oms_type=OmsType.HEDGING,  # Venue will generate position IDs
    account_type=AccountType.MARGIN,
    base_currency=None,  # Multi-currency account
    starting_balances=[Money(1_000_000, USD), Money(10_000_000, JPY)],
    fill_model=fill_model,
    modules=[fx_rollover_interest],
)

Now we can add instruments and data. For this backtest we'll pre-process bid and ask side bar data into quote ticks using a `QuoteTickDataWrangler`.

## Configure strategy

Next we'll configure and initialize a simple `EMACross` strategy we'll use for the backtest.

In [ ]:
from nautilus_trader.examples.strategies.ema_cross import EMACross
from nautilus_trader.examples.strategies.ema_cross import EMACrossConfig

# Configure your strategy
config = EMACrossConfig(
    instrument_id=USDJPY_SIM.id,
    bar_type=BarType.from_str("USD/JPY.SIM-5-MINUTE-BID-INTERNAL"),
    fast_ema_period=10,
    slow_ema_period=20,
    trade_size=Decimal(1_000_000),
)

# Instantiate and add your strategy
strategy = EMACross(config=config)
# engine.add_strategy(strategy=strategy)

In [ ]:
# from nautilus_trader.examples.strategies.subscribe import SubscribeStrategy
# from nautilus_trader.examples.strategies.subscribe import SubscribeStrategyConfig

# # Instantiate your strategy
# class PSubscribeStrategy(SubscribeStrategy):
#     def __init__(self, config: SubscribeStrategyConfig) -> None:
#         super().__init__(config)

#     def on_start(self) -> None:
#         """
#         Actions to be performed on strategy start.
#         """
#         self.instrument = self.cache.instrument(self.instrument_id)
#         if self.instrument is None:
#             self.log.error(f"Could not find instrument for {self.instrument_id}")
#             self.stop()
#             return

#         if self.config.book_type:
#             self.book = OrderBook(
#                 instrument_id=self.instrument.id,
#                 book_type=self.config.book_type,
#             )
#             if self.config.snapshots:
#                 self.subscribe_order_book_at_interval(
#                     instrument_id=self.instrument_id,
#                     book_type=self.config.book_type,
#                 )
#             else:
#                 self.subscribe_order_book_deltas(
#                     instrument_id=self.instrument_id,
#                     book_type=self.config.book_type,
#                 )

#         if self.config.trade_ticks:
#             self.subscribe_trade_ticks(instrument_id=self.instrument_id)
#         if self.config.quote_ticks:
#             self.subscribe_quote_ticks(instrument_id=self.instrument_id)
#         if self.config.bars:
#             bar_type: BarType = BarType(
#                 instrument_id=self.instrument_id,
#                 bar_spec=BarSpecification(
#                     step=1,
#                     aggregation=BarAggregation.MINUTE,
#                     price_type=PriceType.LAST,
#                 ),
#                 aggregation_source=AggregationSource.EXTERNAL,
#             )
#             self.subscribe_bars(bar_type)

#     def on_bar(self, bar: Bar) -> None:
#         print("bar => ", bar)


# # Configure your strategy
# # strategy_config = SubscribeStrategyConfig(
# #     instrument_id=InstrumentId.from_str(f"Step Index.{BROKER_SERVER}"),  # "EUR/USD.{BROKER_SERVER}"
# #     quote_ticks=True,
# #     bars=True,
# # )

# strategy = PSubscribeStrategy(config=strategy_config)

In [ ]:
from nautilus_trader.common.enums import LogColor
from nautilus_trader.config import StrategyConfig
from nautilus_trader.core.correctness import PyCondition
from nautilus_trader.indicators.ta_lib.manager import TAFunctionWrapper
from nautilus_trader.indicators.ta_lib.manager import TALibIndicatorManager
from nautilus_trader.model.data import Bar
from nautilus_trader.model.data import BarType
from nautilus_trader.model.instruments import Instrument
from nautilus_trader.trading.strategy import Strategy


class TALibStrategyConfig(StrategyConfig, frozen=True):
    """
    Configuration for ``TALibStrategy`` instances.

    Parameters
    ----------
    bar_type : BarType
        The bar type for the strategy.

    """

    bar_type: BarType



class TALibStrategy(Strategy):
    """
    A trading strategy demonstration using TA-Lib (Technical Analysis Library) for
    generating trading signals based on technical indicators. This strategy is intended
    for educational purposes and does not execute real trading orders. Instead, it logs
    potential actions derived from technical analysis signals.

    This strategy is configured to use a variety of technical indicators such as EMA (Exponential
    Moving Averages), RSI (Relative Strength Index), and MACD (Moving Average Convergence Divergence).
    It demonstrates how these indicators can be utilized to identify potential trading opportunities
    based on market data.

    The strategy responds to incoming bar data (candlestick data) and analyzes it using the set
    indicators to make decisions. It can identify conditions like EMA crossovers, overbought or
    oversold RSI levels, and MACD histogram values to log potential buy or sell signals.

    Parameters
    ----------
    config : TALibStrategyConfig
        The configuration object for the strategy, which includes the `bar_type` specifying the
        market data type (like minute bars, tick bars, etc.) to be used in the strategy.

    Attributes
    ----------
    instrument_id : InstrumentId
        The ID of the instrument (like a stock or currency pair) that the strategy operates on.
    bar_type : BarType
        The type of market data bars the strategy is configured to use.
    indicator_manager : TALibIndicatorManager
        Manages the indicators used in the strategy, handling their initialization, update,
        and value retrieval.

    """

    def __init__(self, config: TALibStrategyConfig) -> None:
        PyCondition.type(config.bar_type, BarType, "config.bar_type")
        super().__init__(config)

        # Configuration
        self.instrument_id = config.bar_type.instrument_id
        self.bar_type = config.bar_type

        # Create the indicators for the strategy
        self.indicator_manager: TALibIndicatorManager = TALibIndicatorManager(
            bar_type=self.bar_type,
            period=2,
        )

        # Specify the necessary indicators, configuring them as individual or grouped instances
        # in TALibIndicatorManager.  This approach uses string identifiers, each corresponding to
        # an indicator's output name, to instantiate TAFunctionWrappers
        indicators = [
            "ATR_14",
            "EMA_10",
            "EMA_20",
            "RSI_14",
            "MACD_12_26_9",
            "MACD_12_26_9_SIGNAL",
            "MACD_12_26_9_HIST",
        ]
        self.indicator_manager.set_indicators(TAFunctionWrapper.from_list_of_str(indicators))

        # Initialize on_start
        self.instrument: Instrument | None = None

    def on_start(self) -> None:
        """
        Actions to be performed on strategy start.
        """
        self.instrument = self.cache.instrument(self.instrument_id)
        if self.instrument is None:
            self.log.error(f"Could not find instrument for {self.instrument_id}")
            self.stop()
            return

        # Register the indicators for updating
        self.register_indicator_for_bars(self.bar_type, self.indicator_manager)

        # Subscribe to live data
        self.subscribe_bars(self.bar_type)
        self.subscribe_quote_ticks(self.instrument_id)

    def on_bar(self, bar: Bar) -> None:
        """
        Actions to be performed when the strategy is running and receives a bar.

        Parameters
        ----------
        bar : Bar
            The bar received.

        """
        self.log.info(repr(bar), LogColor.CYAN)

        # Check if indicators ready
        if not self.indicators_initialized():
            self.log.info(
                f"Waiting for indicators to warm up [{self.cache.bar_count(self.bar_type)}]",
                color=LogColor.BLUE,
            )
            return  # Wait for indicators to warm up...

        if bar.is_single_price():
            # Implies no market information for this bar
            return

        # Check EMA cross-over
        if self.indicator_manager.value("EMA_10") > self.indicator_manager.value(
            "EMA_20",
            1,
        ) and self.indicator_manager.value("EMA_10", 1) < self.indicator_manager.value("EMA_20"):
            self.log.info("EMA_10 crossed above EMA_20", color=LogColor.GREEN)
        elif self.indicator_manager.value("EMA_10") < self.indicator_manager.value(
            "EMA_20",
            1,
        ) and self.indicator_manager.value("EMA_10", 1) > self.indicator_manager.value("EMA_20"):
            self.log.info("EMA_10 crossed below EMA_20", color=LogColor.GREEN)

        # Check RSI
        if self.indicator_manager.value("RSI_14") > 70:
            self.log.info("RSI_14 is overbought", color=LogColor.MAGENTA)
        elif self.indicator_manager.value("RSI_14") < 30:
            self.log.info("RSI_14 is oversold", color=LogColor.MAGENTA)

        # Check MACD Histogram
        if self.indicator_manager.value("MACD_12_26_9_HIST") > 0:
            self.log.info("MACD_12_26_9_HIST is positive", color=LogColor.MAGENTA)
        elif self.indicator_manager.value("MACD_12_26_9_HIST") < 0:
            self.log.info("MACD_12_26_9_HIST is negative", color=LogColor.MAGENTA)

    def on_stop(self) -> None:
        """
        Actions to be performed when the strategy is stopped.
        """
        # Unsubscribe from data
        self.unsubscribe_bars(self.bar_type)



## Generating reports

Additionally, we can produce various reports to further analyze the backtest result.

In [ ]:
engine.trader.generate_account_report(SIM)

In [ ]:
engine.trader.generate_order_fills_report()

In [ ]:
engine.trader.generate_positions_report()

In [ ]:
# env = gym.make("CartPole-v1", render_mode="human")

# model = PPO("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=10_000)

# vec_env = model.get_env()
# obs = vec_env.reset()
# for i in range(1000):
#     action, _states = model.predict(obs, deterministic=True)
#     obs, reward, done, info = vec_env.step(action)
#     vec_env.render()
#     # VecEnv resets automatically
#     # if done:
#     #   obs = env.reset()

# env.close()